In [1]:
%pylab inline
import pandas as pd
import urllib
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
import numpy as np


from sklearn import neural_network
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn import neighbors
from sklearn import tree
from sklearn.metrics import confusion_matrix
from random import randrange
import matplotlib.pyplot as plt
import h5py
from sklearn import preprocessing

plt.style.use('bmh')

#%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


### Chargement des données d'apprentissage 

In [2]:
def proportion_classe(y_train):
    return len(y_train[y_train["label"]==0])/len(y_train), len(y_train[y_train["label"]==1])/len(y_train), len(y_train[y_train["label"]==2])/len(y_train), len(y_train[y_train["label"]==3])/len(y_train),len(y_train[y_train["label"]==4])/len(y_train)

In [3]:
#Récupération de y_train
output_train=pd.read_csv("output_training.csv", sep=";")

In [4]:
#il y a 11.2% de cycle 0; 3.1% de cycle 1; 36.8% de cycle 2; 31.4% de cycle 3 et 17.4% de cycle 4
proportion_classe(output_train)

(0.11268537531371207,
 0.031006160164271046,
 0.36821811544604155,
 0.31439653205566964,
 0.17369381702030573)

In [5]:
def base_recup_key(fichier,key,output_train,normalize=False):
    
    h5 = h5py.File(fichier, "r")
    
    if normalize==True:
        min_max_scaler = preprocessing.MinMaxScaler()
        input_train=min_max_scaler.fit_transform(h5[key][:])
    else:
        input_train=pd.DataFrame(h5[key][:])

    base_train=pd.concat([input_train,output_train["label"]],axis=1)
    #print(np.shape(base_train))
    
    return base_train

In [6]:
def score(cm,y_test_1):
    score=0
    for i in range(len(cm)):
        for j in range(len(cm)):
            if i!=j:
                score+=cm[i,j]
    return 1-score/len(y_test_1)

In [7]:
def var_importante(base,prop_train,nb_var):
    
    base_train=base[:int(len(base)*prop_train)]
    base_test=base[int(len(base)*prop_train):]
    print("train",proportion_classe(base_train))
    print("test",proportion_classe(base_test))
    
    X_train=base_train.drop("label",axis=1)
    X_test=base_test.drop("label",axis=1)
    y_train=base_train["label"]
    y_test=base_test["label"]
    print(np.shape(X_train),np.shape(X_test),np.shape(y_train),np.shape(y_test))

    modele=ensemble.RandomForestClassifier(n_estimators=100,n_jobs=2)
    modele.fit(X_train,y_train)
    y_pred=modele.predict(X_test)
    cm=confusion_matrix(y_test,y_pred)
    #print(cm)
    
    #importance des variables 
    feature_importance = modele.feature_importances_
    sorted_idx = np.argsort(feature_importance)[-nb_var:]
    print("score",score(cm,y_test))

    return score(cm,y_test),modele ,sorted_idx

In [8]:
#6s pour récupérer 43 000 * 3751 data
t0=time.time()
eeg_1=base_recup_key("train.h5","eeg_1",output_train)
t1=time.time()
print(t1-t0)

5.18175482749939


In [12]:
np.shape(eeg_1)[1]*0.1

375.1

In [10]:
t0=time.time()
s_1=var_importante(eeg_1[:1000],0.75,100)
t1=time.time()
print(t1-t0)

train (0.07466666666666667, 0.02266666666666667, 0.23733333333333334, 0.368, 0.29733333333333334)
test (0.176, 0.084, 0.216, 0.52, 0.004)
(750, 3750) (250, 3750) (750,) (250,)
score 0.548
2.058682680130005


In [ ]:
feature_name = eeg_1.columns
limit = 20
feature_importance = s_1[1].feature_importances_[:]
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)[-limit:]
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 1, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, feature_name[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')

### Identification des variables importantes pour chaque base

In [ ]:
#But: construire une base regroupant les 900 variables les plus importantes

In [15]:
list_keys=['eeg_1', 'eeg_2', 'eeg_3', 'eeg_4', 'po_ir', 'po_r', 'accelerometer_x', 'accelerometer_y', 'accelerometer_z']

In [13]:
def construction_base(list_keys,pourcentage_var,nb_obs,output_train):
    dic_base={}
    for k in list_keys:
        print(k)
        b=base_recup_key("train.h5",k,output_train)
        v_i=var_importante(b[:nb_obs],0.75,int(pourcentage_var*np.shape(b)[1]))
        dic_base[k]=b.iloc[:,v_i[2]]

    base_finale=pd.concat([dic_base[k] for k in list_keys],axis=1,ignore_index=True)
    
    return pd.concat([base_finale,output_train["label"]],axis=1)
    

In [18]:
#on garde 10% des meilleures variables de la prédiction sur chaque dataset 
pourcentage_var=0.1
#il s'agit du nb d'observations pour faire tourner le modèle sur chaque sous-base
nb_obs=5000
t0=time.time()
base_train=construction_base(list_keys,pourcentage_var,nb_obs,output_train)
t1=time.time()
print(t1-t0)

eeg_1
train (0.07253333333333334, 0.03626666666666667, 0.3661333333333333, 0.3232, 0.20186666666666667)
test (0.1728, 0.0208, 0.392, 0.2584, 0.156)
(3750, 3750) (1250, 3750) (3750,) (1250,)
score 0.5016
eeg_2
train (0.07253333333333334, 0.03626666666666667, 0.3661333333333333, 0.3232, 0.20186666666666667)
test (0.1728, 0.0208, 0.392, 0.2584, 0.156)
(3750, 3750) (1250, 3750) (3750,) (1250,)
score 0.516
eeg_3
train (0.07253333333333334, 0.03626666666666667, 0.3661333333333333, 0.3232, 0.20186666666666667)
test (0.1728, 0.0208, 0.392, 0.2584, 0.156)
(3750, 3750) (1250, 3750) (3750,) (1250,)
score 0.4064
eeg_4
train (0.07253333333333334, 0.03626666666666667, 0.3661333333333333, 0.3232, 0.20186666666666667)
test (0.1728, 0.0208, 0.392, 0.2584, 0.156)
(3750, 3750) (1250, 3750) (3750,) (1250,)
score 0.4552
po_ir
train (0.07253333333333334, 0.03626666666666667, 0.3661333333333333, 0.3232, 0.20186666666666667)
test (0.1728, 0.0208, 0.392, 0.2584, 0.156)
(3750, 1500) (1250, 1500) (3750,) (1250,)

In [19]:
base_train.head()

,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,label
0,3.049927,-11.093723,-4.177071,19.371773,-22.776264,17.928631,-19.091215,-10.968331,-9.259424,-12.380516,...,0.000018,-0.000180,0.000391,0.000223,-0.000125,0.000035,0.000203,0.000315,-0.000174,0
1,-37.433933,3.758461,-3.054089,-29.163702,47.254265,-8.906521,-13.863186,1.052138,-1.555163,-29.062410,...,0.000289,0.000119,-0.000204,-0.000023,0.000025,-0.000006,-0.000019,0.000158,0.000020,0
2,-9.764781,-27.430002,2.551445,-12.432044,-12.063060,28.940365,12.601622,-24.994949,-18.387829,-13.569963,...,0.000239,-0.000352,0.000053,0.000005,0.000299,0.000022,0.000007,0.000072,0.000292,0
3,-3.914347,2.692587,79.888214,-556.506653,-40.213840,-42.984577,180.308090,194.375656,-48.523567,843.039246,...,-0.000059,0.012904,0.020396,0.006740,-0.006698,0.006905,0.006763,-0.010783,-0.005100,0
4,48.261055,-151.069962,102.285561,2.079795,-24.304810,-64.906761,231.104996,-77.054688,46.936745,-8.382654,...,0.000209,-0.000187,0.000141,-0.000164,0.000013,-0.000082,-0.000155,-0.000420,-0.000054,0


In [21]:
np.shape(base_train)

(43830, 2251)

### Récupération des 10 000 premières lignes (à jeter aussi mais refaire le truc pour récupérer data test)

In [ ]:
# La représentation est assez fidèles 
proportion_classe(output_train[:10000])

In [ ]:
np.concatenate?

In [ ]:
def base_recup(fichier,num_depart,num_final,nb_var,output_train,normalize=True):
    dic_train={}
    h5 = h5py.File(fichier, "r")
    min_max_scaler = preprocessing.MinMaxScaler()
    for k in list_keys:
        t2=time.time()
        print(k)
        if normalize==True:
            dic_train[k]=min_max_scaler.fit_transform(h5[k][num_depart:num_final,:nb_var])
        else:
            dic_train[k]=h5[k][num_depart:num_final,:nb_var]
        t3=time.time()
        print(t3-t2)
        print(np.shape(dic_train[k]))
    X_train=pd.DataFrame(np.concatenate([dic_train[k] for k in list_keys],axis=1))
    t4=time.time()
    print(t4-t3)
    #print(np.shape(X_train))
    if fichier=='train.h5':
        #print(y_train[num_depart:num_final]["label"].reset_index())
        base_train=pd.concat([X_train,output_train["label"][num_depart:num_final].reset_index()],axis=1)
        return base_train.sample(frac=1).reset_index().drop(["index","level_0"],axis=1)
    else:
        return X_train
        
    #print(np.shape(base_train))
    
    #return base_train.sample(frac=1).reset_index().drop(["index","level_0"],axis=1)

In [ ]:
#on récupère une sous_base de 10 000 observations 
#par contre on récupère les 10 000 premières ...
t0=time.time()
base_1=base_recup('train.h5',0,1000,100,output_train,normalize=False)
t1=time.time()
print(t1-t0)

In [ ]:
base_1.tail()

### Cross validation

In [22]:
#On associe à chaque base de test sa base de train correspondante
def CV(base,nb_CV):
            
    nb=int(len(base)/nb_CV)
    print(nb)
    separ_test=[base[i*nb:(i+1)*nb] for i in range (nb_CV)]
    #print(len(separ_test))

    separ_train=[]
    list_train=[[] for p in range(nb_CV)]
    for i in range(nb_CV):
        l=[j for j in range(nb_CV)]
        l.remove(i)
        #print(l)
        for p in l:
            list_train[i].append(separ_test[p])
        separ_train.append(pd.concat(list_train[i]))
        
    return (separ_train,separ_test)

In [23]:
def choix_estimateur(modele, bases_CV,nb_CV, param_1,param_2, RFC=True,LR=False):
    l_res={}
    for i in param_1:
        l_res[i]={}
        for j in param_2:
            l_res[i][j]=[]
    for i in range(nb_CV):
        print(i)
        X_train=bases_CV[0][i].drop("label",axis=1)
        y_train=bases_CV[0][i]["label"]
        X_test=bases_CV[1][i].drop("label",axis=1)
        y_test=bases_CV[1][i]["label"]
        print(np.shape(X_train),np.shape(X_test),np.shape(y_train),np.shape(y_test))

        for j in param_1:
            print(j)
            for p in param_2:
                print(p)
                if RFC==True:
                    mod=modele(n_estimators=j,min_samples_leaf=p,n_jobs=2)

                elif LR==True:
                    mod=modele(solver=j)
                else:
                    mod=modele(n_neighbors=j)
                t2=time.time()    
                mod=mod.fit(X_train,y_train)
                y_pred=mod.predict(X_test)
                t3=time.time()
                print(t3-t2)
                cm=confusion_matrix(y_test,y_pred)
                #print(cm)
                l_res[j][p].append(score(cm,y_test))
    return l_res

In [ ]:
#on est sur 5000 obs et 2250 variables et on fait une CV sur 5 sous-bases
nb_tree=[20,50,100,150,200]
min_leaf=[1,2,3,5]
t0=time.time()
est_RFC=choix_estimateur(ensemble.RandomForestClassifier,CV(base_train[:5000],5),5,nb_tree,min_leaf)
t1=time.time()
print(t1-t0)


1000
0
(4000, 2250) (1000, 2250) (4000,) (1000,)
20
1
2.5938820838928223
2
2.2902731895446777
3
2.202880859375
5
2.0884180068969727
50
1
5.627327919006348
2
5.317632675170898
3
5.322713136672974
5
5.4130308628082275
100
1
12.465969800949097
2
11.082095861434937
3
10.425899982452393
5
10.563969612121582
150
1
17.609516143798828
2
17.79293179512024
3
16.793725967407227
5
16.85985279083252
200
1
25.8250150680542
2
24.596960067749023
3
22.89516592025757
5
23.57281994819641
1
(4000, 2250) (1000, 2250) (4000,) (1000,)
20
1
3.225529909133911
2
2.8044958114624023
3
2.4144208431243896
5
2.3125360012054443
50
1
7.199557065963745
2
6.23526406288147
3
5.829571008682251
5
5.524724960327148
100
1
12.609413146972656
2
12.320431232452393
3
11.128785133361816
5
10.724411964416504
150
1
20.3857421875
2
18.008984088897705
3
16.892704963684082
5
16.280193090438843
200
1
23.756004095077515
2
22.828533172607422
3
23.950608015060425
5
22.110790967941284
2
(4000, 2250) (1000, 2250) (4000,) (1000,)
20
1
2.4155

In [ ]:
#faire de beaux graphiques

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10, 5))

colors=["red","green","blue","orange"]

#ax1.plot(,[1-np.mean(est_RFC_900_var[i]) for i in nb_tree],color="green",label="taux d'erreur 900 var")
ax1.plot([i for i in nb_tree],[1-np.mean(est_RFC_9000_var[i][min_leaf[j]]) for i in nb_tree],color=colors[j],label="taux d'erreur min leaf"+str(i))
ax2.plot([i for i in nb_tree],[np.std(est_RFC_9000_var[i][min_leaf[j]]) for i in nb_tree],color=colors[j],label="variance des erreurs min leaf"+str(i))


ax1.set_title("erreur RFC")
ax1.set_ylim(0.3,0.5)
ax1.set_xlabel("nombre d'arbres")
ax1.set_ylabel("erreur")
ax1.legend(loc=0)

ax2.set_title("variance RFC")
ax2.set_ylim(0,0.2)
ax2.set_xlabel("nombre d'arbres")
ax2.set_ylabel("variance")
ax2.legend(loc=0)

### Récupération données test et prédiction finale

In [ ]:
t0=time.time()
data_test=base_recup("test.h5",0,20592,output_train,normalize=False)
t1=time.time()
print(t1-t0)

In [ ]:
np.shape(data_test)

In [ ]:
#PB: si je fais comme ça, j'applique différentes normalisations aux dataset ...
#t0=time.time
#bases_comp=[]
#for i in range(10):
#    print(i)
#    bases_comp.append(base_recup("train.h5",i*4383, (i+1)*4383,output_train,normalize=False))
##base_complete=pd.concat(bases_comp,axis=0)
#


In [ ]:
t0=time.time()
base_complete=base_recup("train.h5",5000,25000,output_train,normalize=False)
t1=time.time()
print(t1-t0)

In [ ]:
base_complete.head()

In [ ]:
t0=time.time()
X_train=base_complete.drop("label",axis=1)
y_train=base_complete["label"]
modele=ensemble.RandomForestClassifier(n_estimators=200)
modele.fit(X_train,y_train)
t1=time.time()
print(t1-t0)
y_pred=modele.predict(data_test)

In [ ]:
#Créer un csv avec le résultat
y_pred

In [ ]:
base_pred=pd.DataFrame(y_pred,columns=["label"])

In [ ]:
#base_pred.to_csv("pred_2.csv",index_label="id",sep=";")

### Comparaison avec d'anciennes prédictions 

In [ ]:
base_pred_1=pd.read_csv("pred_1.csv",sep=";")

In [ ]:
base_pred_1.head()

In [ ]:
base_pred_1["diff"]=base_pred_1["label"]-base_pred["label"]

In [ ]:
len(base_pred_1[base_pred_1["diff"]==0])